# High-level Comparison Between the Network Covariance and Characteristic Matrices

In this notebook, we provide a rough comparison between covariance-based and entropy-based topology inference methods.
The goal is to test each method across a range of entangled states to get an idea of their relative performance.


## Key Differences

### Covariance Matrix Advantages

There is a significant performance advantage over the characteristic matrix:
* This can easily be observed by noting the cell execution times, especially when more qubits are considered. 
* The covariance is evaluated from one circuit evaluation $O(1)$, whereas the charcteristic matrix requires a minimum of $O(n^2)$ circuit evaluations where $n$ is the number of qubits. Given clever parallelization in quantum circuits, the characteristic matrix may be evaluated in $O(n)$.
* Optimizing the covariance matrix seems to require significantly fewer steps than needed to successfully infer topology using the characteristic matrix. This means that fewer circuits need to be run.
* The optimization appears to be more robust and less susceptible to local minima. We see this in the fact that the characteristic matrix often fails to infer the GHZ states with more than 3 qubits.


### Characteristic Matrix Advantages

A key difference between the covariance and characteristic matrices is that the covariance matrix seems to always have ones along the diagonal, even if the measured qubit is not correlated with any other qubits. On the contrary, the characteristic matrix measures a zero von neumann entropy if there is no correlation whatsoever.

The threshold of deciding if a correlation exists needs to be raised as the number of shots decreases.
The rate at which covariance methods need to be raised seems to more than characteristic matrices.
This may mean, that the variance of the entropic quantities in the low shot regime are improved over those of the covariance matrix.

In [1]:
import qnetti
import pennylane as qml
from pennylane import numpy as qnp
import qnetvo

print(qnetvo.__version__)
print(qml.__version__)

0.4.1
0.28.0


In [2]:
def decision_matrix(mat, atol=0.1):
    return qnp.where(qnp.abs(mat) > atol, 1, 0)


def matrix_distance(mat1, mat2):
    return qnp.linalg.norm(qnp.array(mat1) - qnp.abs(qnp.array(mat2)))


def characteristic_matrix_inference(prep_node, expected_mat, **kwargs):
    char_mat = qnetti.qubit_characteristic_matrix(prep_node, **kwargs)
    dec_mat = decision_matrix(char_mat)
    char_dist = matrix_distance(expected_mat, char_mat)

    print("characteristic matrix :\n", char_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", char_dist)


def covariance_matrix_inference(
    prep_node,
    num_qubits,
    expected_mat,
    meas_wires=None,
    step_size=0.1,
    num_steps=10,
    verbose=False,
    shots=None,
    qnode_kwargs={},
    atol=0.1,
):
    settings = qnp.random.rand(3 * num_qubits, requires_grad=True)
    cov_cost = qnetti.qubit_covariance_cost_fn(
        prep_node, meas_wires=meas_wires, shots=shots, qnode_kwargs=qnode_kwargs
    )

    settings_list = [settings]
    cost_vals = [cov_cost(settings)]

    opt = qml.GradientDescentOptimizer(stepsize=step_size)
    for i in range(num_steps):
        settings, cost_val = opt.step_and_cost(cov_cost, settings)

        cost_vals += [cost_val]
        settings_list += [settings]

    min_id = qnp.argmin(cost_vals)
    min_cost = cost_vals[min_id]
    opt_settings = settings_list[min_id]

    cov_mat = qnetti.qubit_covariance_matrix_fn(
        prep_node, meas_wires=meas_wires, shots=shots, qnode_kwargs=qnode_kwargs
    )(opt_settings)
    dec_mat = decision_matrix(cov_mat, atol=atol)
    cov_dist = matrix_distance(expected_mat, cov_mat)

    print("covariance matrix :\n", cov_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", cov_dist)

## Two-Qubit GHZ State

In [3]:
ghz2_mat = qnp.ones((2, 2))


def rot_bell_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])


bell_state_prep_node = qnetvo.PrepareNode(wires=[0, 1], ansatz_fn=rot_bell_state)

In [4]:
characteristic_matrix_inference(bell_state_prep_node, ghz2_mat, num_steps=10, step_size=0.1)

characteristic matrix :
 [[1.         0.78365207]
 [0.78365207 1.        ]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  0.3059621710953783


In [5]:
covariance_matrix_inference(
    bell_state_prep_node,
    2,
    ghz2_mat,
    step_size=0.2,
    num_steps=6,
    verbose=False,
)

covariance matrix :
 [[1.         0.99999249]
 [0.99999249 1.        ]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  1.0624792979292031e-05


## 3-Qubit GHZ state

In [6]:
ghz3_mat = qnp.ones((3, 3))


def rot_ghz3_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])


ghz3_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2], ansatz_fn=rot_ghz3_state)

In [7]:
characteristic_matrix_inference(ghz3_state_prep_node, ghz3_mat, num_steps=10, step_size=0.1)

characteristic matrix :
 [[1.         0.39974909 0.07939856]
 [0.39974909 1.         0.025727  ]
 [0.07939856 0.025727   1.        ]]
decision matrix :
 [[1 1 0]
 [1 1 0]
 [0 0 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
distance to expected :  2.0770248184914895


In [8]:
covariance_matrix_inference(
    ghz3_state_prep_node,
    3,
    ghz3_mat,
    step_size=0.1,
    num_steps=10,
    verbose=False,
)

covariance matrix :
 [[ 1.         -0.99903341  0.99904189]
 [-0.99903341  1.         -0.99999144]
 [ 0.99904189 -0.99999144  1.        ]]
decision matrix :
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
distance to expected :  0.0019247481672651323


## 4-Qubit GHZ

In [9]:
ghz4_mat = qnp.ones((4, 4))


def rot_ghz4_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])


ghz4_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3], ansatz_fn=rot_ghz4_state)

In [10]:
characteristic_matrix_inference(ghz4_state_prep_node, ghz4_mat, num_steps=30, step_size=0.1)

characteristic matrix :
 [[1.         0.56472518 0.15404376 0.59222568]
 [0.56472518 1.         0.40370309 0.33911379]
 [0.15404376 0.40370309 1.         0.95743003]
 [0.59222568 0.33911379 0.95743003 1.        ]]
decision matrix :
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
distance to expected :  1.9315997552287274


In [11]:
covariance_matrix_inference(
    ghz4_state_prep_node,
    4,
    ghz4_mat,
    step_size=0.2,
    num_steps=6,
    verbose=False,
)

covariance matrix :
 [[ 1.         -0.99554295  0.99349685 -0.89565097]
 [-0.99554295  1.         -0.9946274   0.89667017]
 [ 0.99349685 -0.9946274   1.         -0.89482729]
 [-0.89565097  0.89667017 -0.89482729  1.        ]]
decision matrix :
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
distance to expected :  0.2558049809093404


# 5-Qubit GHZ

In [12]:
ghz5_mat = qnp.ones((5, 5))


def rot_ghz5_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])


ghz5_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3, 4], ansatz_fn=rot_ghz5_state)

In [13]:
characteristic_matrix_inference(ghz5_state_prep_node, ghz5_mat, num_steps=30, step_size=0.1)

characteristic matrix :
 [[1.00000000e+00 3.25660236e-03 4.05698139e-04 9.38272283e-01
  9.99669349e-01]
 [3.25660236e-03 1.00000000e+00 6.29563962e-01 4.06433674e-01
  2.90222067e-01]
 [4.05698139e-04 6.29563962e-01 1.00000000e+00 9.81897892e-01
  9.95862592e-01]
 [9.38272283e-01 4.06433674e-01 9.81897892e-01 1.00000000e+00
  9.23585631e-01]
 [9.99669349e-01 2.90222067e-01 9.95862592e-01 9.23585631e-01
  1.00000000e+00]]
decision matrix :
 [[1 0 0 1 1]
 [0 1 1 1 1]
 [0 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  2.447859924862354


In [14]:
covariance_matrix_inference(
    ghz5_state_prep_node, 5, ghz5_mat, step_size=0.1, num_steps=10, verbose=False
)

covariance matrix :
 [[ 1.          0.53899521  0.9999711  -0.99131748 -0.99996745]
 [ 0.53899521  1.          0.53901079 -0.53434626 -0.53900882]
 [ 0.9999711   0.53901079  1.         -0.99134613 -0.99999635]
 [-0.99131748 -0.53434626 -0.99134613  1.          0.99134252]
 [-0.99996745 -0.53900882 -0.99999635  0.99134252  1.        ]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  1.3073698562073244


# 8-qubit GHZ state

In [15]:
ghz8_mat = qnp.ones((8, 8))


def rot_ghz8_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[5]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[6]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[7]])


ghz8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=rot_ghz8_state,
)

In [16]:
characteristic_matrix_inference(ghz8_state_prep_node, ghz8_mat, num_steps=30, step_size=0.1)

characteristic matrix :
 [[1.         1.         0.39801406 0.53690655 0.99999999 0.23410821
  0.9419895  1.        ]
 [1.         1.         1.         0.99958445 0.99929754 0.87873912
  0.90108223 0.00279393]
 [0.39801406 1.         1.         0.02193399 0.94614372 0.95539924
  0.02311805 0.00307238]
 [0.53690655 0.99958445 0.02193399 1.         0.52315375 0.97566352
  0.89143051 0.93771627]
 [0.99999999 0.99929754 0.94614372 0.52315375 1.         1.
  0.90386864 0.76105892]
 [0.23410821 0.87873912 0.95539924 0.97566352 1.         1.
  0.93392805 1.        ]
 [0.9419895  0.90108223 0.02311805 0.89143051 0.90386864 0.93392805
  1.         0.68314503]
 [1.         0.00279393 0.00307238 0.93771627 0.76105892 1.
  0.68314503 1.        ]]
decision matrix :
 [[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0]
 [1 1 1 0 1 1 0 0]
 [1 1 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 0 1 1 1 1 1]
 [1 0 0 1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1

In [17]:
covariance_matrix_inference(
    ghz8_state_prep_node,
    8,
    ghz8_mat,
    step_size=0.1,
    num_steps=10,
    verbose=False,
)

covariance matrix :
 [[ 1.          0.93545218  0.91636829 -0.91293008 -0.897829    0.89101795
  -0.79674341 -0.92193694]
 [ 0.93545218  1.          0.94658663 -0.94303505 -0.92743599  0.92040033
  -0.82301699 -0.95233891]
 [ 0.91636829  0.94658663  1.         -0.92379646 -0.90851563  0.90162351
  -0.80622685 -0.93291052]
 [-0.91293008 -0.94303505 -0.92379646  1.          0.90510689 -0.89824062
   0.80320189  0.92941025]
 [-0.897829   -0.92743599 -0.90851563  0.90510689  1.         -0.88338253
   0.78991586  0.91403656]
 [ 0.89101795  0.92040033  0.90162351 -0.89824062 -0.88338253  1.
  -0.78392346 -0.90710256]
 [-0.79674341 -0.82301699 -0.80622685  0.80320189  0.78991586 -0.78392346
   1.          0.81112619]
 [-0.92193694 -0.95233891 -0.93291052  0.92941025  0.91403656 -0.90710256
   0.81112619  1.        ]]
decision matrix :
 [[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
expect

# 8 qubit graph state

In [18]:
# This expected matrix is not known with certainty, but agrees with results
graph8_mat = qnp.array(
    [
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [0, 3], [6, 7]]),
)

In [19]:
characteristic_matrix_inference(graph8_state_prep_node, graph8_mat, num_steps=60, step_size=0.02)

characteristic matrix :
 [[1.00000000e+00 2.66453526e-15 3.55271368e-15 7.87658324e-01
  2.66453526e-15 3.99680289e-15 3.10862447e-15 1.55431223e-15]
 [2.66453526e-15 1.00000000e+00 4.21544863e-01 3.10862447e-15
  2.66453526e-15 2.88657986e-15 2.66453526e-15 2.66453526e-15]
 [3.55271368e-15 4.21544863e-01 1.00000000e+00 2.88657986e-15
  2.66453526e-15 3.10862447e-15 2.66453526e-15 3.10862447e-15]
 [7.87658324e-01 3.10862447e-15 2.88657986e-15 1.00000000e+00
  2.66453526e-15 3.33066907e-15 2.88657986e-15 1.55431223e-15]
 [2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
  1.42757031e-05 3.99680289e-15 3.55271368e-15 8.88178420e-16]
 [3.99680289e-15 2.88657986e-15 3.10862447e-15 3.33066907e-15
  3.99680289e-15 8.02966106e-05 2.66453526e-15 3.77475828e-15]
 [3.10862447e-15 2.66453526e-15 2.66453526e-15 2.88657986e-15
  3.55271368e-15 2.66453526e-15 1.00000000e+00 6.02599659e-01]
 [1.55431223e-15 2.66453526e-15 3.10862447e-15 1.55431223e-15
  8.88178420e-16 3.77475828e-15 6.0259

In [20]:
covariance_matrix_inference(
    graph8_state_prep_node,
    8,
    graph8_mat,
    step_size=0.05,
    num_steps=30,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00 -8.32667268e-17 -2.46519033e-32 -9.63371307e-01
  -2.67599372e-17  8.33048101e-17 -2.77555756e-17 -5.55111512e-17]
 [-8.32667268e-17  1.00000000e+00 -9.99997414e-01  2.77555756e-17
  -1.40271336e-16 -5.55682762e-17  1.84889275e-32 -8.32667268e-17]
 [-2.46519033e-32 -9.99997414e-01  1.00000000e+00 -5.55111512e-17
   2.97468524e-17  5.55873178e-17 -2.46519033e-32  2.77555756e-17]
 [-9.63371307e-01  2.77555756e-17 -5.55111512e-17  1.00000000e+00
  -2.92490332e-17 -5.55682762e-17  5.55111512e-17 -2.77555756e-17]
 [-2.67599372e-17 -1.40271336e-16  2.97468524e-17 -2.92490332e-17
   9.99919577e-01  1.48176556e-16 -2.67599372e-17 -1.49345759e-18]
 [ 8.33048101e-17 -5.55682762e-17  5.55873178e-17 -5.55682762e-17
   1.48176556e-16  9.99999882e-01  5.55492345e-17 -5.55682762e-17]
 [-2.77555756e-17  1.84889275e-32 -2.46519033e-32  5.55111512e-17
  -2.67599372e-17  5.55492345e-17  1.00000000e+00 -9.99999405e-01]
 [-5.55111512e-17 -8.32667268e-17  2.77555756e-1

## 8-Qubit Graph State with Frustrated CNOT arrangement

In [21]:
graph8_cyclic_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_cyclic_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [2, 0], [6, 7]]),
)

In [22]:
characteristic_matrix_inference(
    graph8_cyclic_state_prep_node, graph8_cyclic_mat, num_steps=60, step_size=0.02
)

characteristic matrix :
 [[1.00000000e+00 9.89314418e-01 2.08457690e-02 3.10862447e-15
  1.77635684e-15 3.55271368e-15 4.44089210e-15 2.88657986e-15]
 [9.89314418e-01 1.00000000e+00 2.59852705e-03 3.55271368e-15
  2.88657986e-15 2.44249065e-15 4.44089210e-15 4.44089210e-15]
 [2.08457690e-02 2.59852705e-03 1.00000000e+00 3.10862447e-15
  3.77475828e-15 3.10862447e-15 2.88657986e-15 3.10862447e-15]
 [3.10862447e-15 3.55271368e-15 3.10862447e-15 6.60970837e-07
  2.44249065e-15 2.88657986e-15 3.33066907e-15 1.11022302e-15]
 [1.77635684e-15 2.88657986e-15 3.77475828e-15 2.44249065e-15
  9.31404262e-09 1.90125693e-15 2.22044605e-15 2.44249065e-15]
 [3.55271368e-15 2.44249065e-15 3.10862447e-15 2.88657986e-15
  1.90125693e-15 9.57368532e-05 2.88657986e-15 2.44249065e-15]
 [4.44089210e-15 4.44089210e-15 2.88657986e-15 3.33066907e-15
  2.22044605e-15 2.88657986e-15 1.00000000e+00 9.99999995e-01]
 [2.88657986e-15 4.44089210e-15 3.10862447e-15 1.11022302e-15
  2.44249065e-15 2.44249065e-15 9.9999

In [23]:
covariance_matrix_inference(
    graph8_cyclic_state_prep_node,
    8,
    graph8_cyclic_mat,
    step_size=0.05,
    num_steps=30,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00  9.99999999e-01  9.99999998e-01  2.77458591e-17
  -2.77618030e-17 -1.39065887e-16 -3.08148791e-33 -2.77555756e-17]
 [ 9.99999999e-01  1.00000000e+00  9.99999998e-01  8.32667268e-17
   2.77555756e-17 -1.66533454e-16  0.00000000e+00 -2.77555756e-17]
 [ 9.99999998e-01  9.99999998e-01  1.00000000e+00  2.77652921e-17
  -2.77493482e-17 -1.94001021e-16  3.08148791e-33 -2.77555756e-17]
 [ 2.77458591e-17  8.32667268e-17  2.77652921e-17  9.99999969e-01
  -2.05073261e-17 -1.43254447e-19 -5.55208678e-17  2.77750087e-17]
 [-2.77618030e-17  2.77555756e-17 -2.77493482e-17 -2.05073261e-17
   9.99999987e-01  5.55391033e-17  5.55049239e-17 -1.11009848e-16]
 [-1.39065887e-16 -1.66533454e-16 -1.94001021e-16 -1.43254447e-19
   5.55391033e-17  9.99973082e-01 -8.35547356e-17  1.39353896e-16]
 [-3.08148791e-33  0.00000000e+00  3.08148791e-33 -5.55208678e-17
   5.55049239e-17 -8.35547356e-17  1.00000000e+00 -9.99999241e-01]
 [-2.77555756e-17 -2.77555756e-17 -2.77555756e-1

## 12-Qubit Graph State

In [24]:
graph12_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    ]
)
graph12_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [6, 7], [4, 5], [4, 9], [4, 11], [11, 10]]),
)

In [25]:
characteristic_matrix_inference(graph12_state_prep_node, graph12_mat, num_steps=60, step_size=0.02)

characteristic matrix :
 [[ 1.00000000e+00  1.03688023e-03  5.19980043e-01  3.77475828e-15
   3.99680289e-15  4.44089210e-15  4.44089210e-15  4.44089210e-15
   3.55271368e-15  2.88657986e-15  1.33226763e-15  2.22044605e-16]
 [ 1.03688023e-03  1.00000000e+00  3.72356509e-01  4.44089210e-15
   4.21884749e-15  3.10862447e-15  3.33066907e-15  4.44089210e-15
   4.44089210e-15  7.54951657e-15  6.21724894e-15 -3.77475828e-15]
 [ 5.19980043e-01  3.72356509e-01  1.00000000e+00  3.77475828e-15
   2.88657986e-15  5.77315973e-15  2.44249065e-15  2.88657986e-15
   4.21884749e-15  1.33226763e-15  1.33226763e-15  2.22044605e-16]
 [ 3.77475828e-15  4.44089210e-15  3.77475828e-15  5.58374350e-03
   4.21884749e-15  3.55271368e-15  4.21884749e-15  4.21884749e-15
   2.22044605e-15  8.43769499e-15  6.66133815e-16  1.59872116e-14]
 [ 3.99680289e-15  4.21884749e-15  2.88657986e-15  4.21884749e-15
   1.00000000e+00  2.75514555e-03  4.88498131e-15  4.44089210e-15
   7.10542736e-15  1.04659354e-02  5.77315973e-

In [26]:
covariance_matrix_inference(
    graph12_state_prep_node,
    12,
    graph12_mat,
    num_steps=20,
    step_size=0.05,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00 -9.99997992e-01  9.99902348e-01 -2.76980114e-17
  -2.77555756e-16  2.22044605e-16  2.77555756e-17 -8.32667268e-17
   4.17070853e-16  1.66533454e-16  2.77555756e-17 -1.11022302e-16]
 [-9.99997992e-01  1.00000000e+00 -9.99902702e-01  2.76980114e-17
   2.49800181e-16 -2.49800181e-16 -5.55111512e-17  8.32667268e-17
  -4.17070853e-16 -1.38777878e-16  2.77555756e-17  8.32667268e-17]
 [ 9.99902348e-01 -9.99902702e-01  1.00000000e+00  1.11137431e-16
  -2.77555756e-16  2.49800181e-16  2.77555756e-17 -5.55111512e-17
  -1.09547864e-16  1.66533454e-16  8.32667268e-17 -1.38777878e-16]
 [-2.76980114e-17  2.76980114e-17  1.11137431e-16  9.99998925e-01
   2.22044605e-16 -1.94289029e-16 -2.76404473e-17  2.77555756e-17
  -7.06222190e-17 -5.55687154e-17  1.94404158e-16 -1.66418325e-16]
 [-2.77555756e-16  2.49800181e-16 -2.77555756e-16  2.22044605e-16
   1.00000000e+00 -9.99999966e-01 -8.32667268e-17  1.38777878e-16
   5.55111512e-17 -9.99999928e-01  5.55111512e-17  

## A bunch of different entangled states

In [27]:
multi_ent_mat_full = qnp.zeros((8, 8))
multi_ent_mat = qnp.array(
    [
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
    ]
)


def fn(settings, wires):
    qnetvo.shared_coin_flip_state([qnp.pi / 3], wires=[wires[0], wires[1], wires[7]])
    qnetvo.ghz_state([], wires=wires[2:4])
    qnetvo.W_state([], wires=wires[4:7])


multi_ent_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=fn,
)

In [28]:
characteristic_matrix_inference(
    multi_ent_prep_node, multi_ent_mat_full, num_steps=50, step_size=0.1
)

characteristic matrix :
 [[9.84364138e-01 6.62297934e-04 2.22044605e-15 1.77635684e-15
  1.55431223e-15 1.33226763e-15 8.88178420e-16 7.87931532e-01]
 [6.62297934e-04 8.25519491e-01 6.66133815e-16 1.99840144e-15
  1.33226763e-15 1.99840144e-15 1.55431223e-15 1.34583775e-01]
 [2.22044605e-15 6.66133815e-16 1.00000000e+00 9.99999994e-01
  1.99840144e-15 1.77635684e-15 2.22044605e-15 2.44249065e-15]
 [1.77635684e-15 1.99840144e-15 9.99999994e-01 1.00000000e+00
  1.55431223e-15 1.77635684e-15 1.99840144e-15 8.88178420e-16]
 [1.55431223e-15 1.33226763e-15 1.99840144e-15 1.55431223e-15
  9.47078313e-01 3.47499237e-01 3.49796881e-01 1.33226763e-15]
 [1.33226763e-15 1.99840144e-15 1.77635684e-15 1.77635684e-15
  3.47499237e-01 9.49612526e-01 3.26599830e-01 1.55431223e-15]
 [8.88178420e-16 1.55431223e-15 2.22044605e-15 1.99840144e-15
  3.49796881e-01 3.26599830e-01 9.23548542e-01 1.99840144e-15]
 [7.87931532e-01 1.34583775e-01 2.44249065e-15 8.88178420e-16
  1.33226763e-15 1.55431223e-15 1.9984

In [29]:
covariance_matrix_inference(
    multi_ent_prep_node,
    7,
    multi_ent_mat,
    meas_wires=[0, 1, 2, 3, 4, 5, 6],
    num_steps=20,
    step_size=0.05,
    verbose=False,
)

covariance matrix :
 [[ 8.87899182e-01  3.78280336e-01  6.46807828e-17  2.77555756e-17
  -3.25260652e-17 -1.22298005e-16 -5.72458747e-17]
 [ 3.78280336e-01  8.58167342e-01 -2.09058753e-17 -2.77555756e-17
  -1.62196645e-16 -2.12503626e-17 -2.11636264e-16]
 [ 6.46807828e-17 -2.09058753e-17  1.00000000e+00  9.99996938e-01
  -1.52721659e-19  5.50655718e-17  2.69218382e-17]
 [ 2.77555756e-17 -2.77555756e-17  9.99996938e-01  1.00000000e+00
   0.00000000e+00  5.55111512e-17  2.77555756e-17]
 [-3.25260652e-17 -1.62196645e-16 -1.52721659e-19  0.00000000e+00
   9.99992431e-01  6.66274752e-01  6.65659853e-01]
 [-1.22298005e-16 -2.12503626e-17  5.50655718e-17  5.55111512e-17
   6.66274752e-01  9.99935570e-01  6.65307720e-01]
 [-5.72458747e-17 -2.11636264e-16  2.69218382e-17  2.77555756e-17
   6.65659853e-01  6.65307720e-01  9.99774421e-01]]
decision matrix :
 [[1 1 0 0 0 0 0]
 [1 1 0 0 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]]
expected matrix :
 [

## Finite Shot Examples

In [3]:
graph5_mat = qnp.array(
    [
        [1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1],
    ]
)

qnode_kwargs = {
    "diff_method": "parameter-shift",
}

graph5_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [2, 0], [3, 4]]),
)

## 1000 Shots

In [4]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=60,
    step_size=0.02,
    shots=1000,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.99604629 0.00443187 0.99833738 0.00464129 0.003807  ]
 [0.00443187 0.99490417 0.00462623 0.00256056 0.00810105]
 [0.99833738 0.00462623 0.99560686 0.00436574 0.00524864]
 [0.00464129 0.00256056 0.00436574 0.99490417 0.67619065]
 [0.003807   0.00810105 0.00524864 0.67619065 0.99685552]]
decision matrix :
 [[1 0 1 0 0]
 [0 1 0 0 0]
 [1 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.0430287835643504


In [5]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.2,
    num_steps=6,
    shots=1000,
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 9.98704e-01  9.88632e-01  9.94848e-01  1.44000e-03 -1.44000e-03]
 [ 9.88632e-01  9.98556e-01  9.88784e-01 -4.80000e-04  4.80000e-04]
 [ 9.94848e-01  9.88784e-01  9.98976e-01  1.28000e-03 -1.28000e-03]
 [ 1.44000e-03 -4.80000e-04  1.28000e-03  9.98400e-01 -9.98400e-01]
 [-1.44000e-03  4.80000e-04 -1.28000e-03 -9.98400e-01  9.98400e-01]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.024371485305577978


## 500 Shots

In [6]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=500,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.99334181 0.00831194 0.99989612 0.00664245 0.00833094]
 [0.00831194 0.98958752 0.01929609 0.00911956 0.01120093]
 [0.99989612 0.01929609 0.99334181 0.00685853 0.00624924]
 [0.00664245 0.00911956 0.00685853 0.99156977 1.        ]
 [0.00833094 0.01120093 0.00624924 1.         0.99218378]]
decision matrix :
 [[1 0 1 0 0]
 [0 1 0 0 0]
 [1 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  1.9727129747696193


In [7]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.2,
    num_steps=10,
    shots=500,
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 0.998976  0.986848  0.979104  0.035872 -0.035872]
 [ 0.986848  0.998704  0.990992  0.031856 -0.031856]
 [ 0.979104  0.990992  0.999216  0.023888 -0.023888]
 [ 0.035872  0.031856  0.023888  0.999984 -0.999984]
 [-0.035872 -0.031856 -0.023888 -0.999984  0.999984]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.11346292123861436


## 100 Shots

In [8]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=100,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.97095059 0.80448638 0.03036928 0.01962822 0.05983848]
 [0.80448638 0.95804202 0.09677215 0.02398292 0.04986686]
 [0.03036928 0.09677215 0.97095059 0.04350992 0.04181393]
 [0.01962822 0.02398292 0.04350992 0.95067209 1.        ]
 [0.05983848 0.04986686 0.04181393 1.         0.95067209]]
decision matrix :
 [[1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  1.9021338163046722


In [10]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    shots=100,
    qnode_kwargs=qnode_kwargs,
    atol=0.3,
)

covariance matrix :
 [[1.     0.98   0.98   0.14   0.14  ]
 [0.98   0.9996 0.9996 0.1212 0.1212]
 [0.98   0.9996 0.9996 0.1212 0.1212]
 [0.14   0.1212 0.1212 0.9964 0.9964]
 [0.14   0.1212 0.1212 0.9964 0.9964]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.44448622025885115


## 50 Shots

In [11]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=50,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.94268319 0.99537844 0.12878805 0.13494474 0.08095388]
 [0.99537844 0.94268319 0.11104761 0.06100533 0.11497788]
 [0.12878805 0.11104761 0.90438146 0.11806366 0.07531321]
 [0.13494474 0.06100533 0.11806366 0.9248187  1.        ]
 [0.08095388 0.11497788 0.07531321 1.         0.90438146]]
decision matrix :
 [[1 1 1 1 0]
 [1 1 1 0 1]
 [1 1 1 1 0]
 [1 0 1 1 1]
 [0 1 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  1.8032406839943667


In [13]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    shots=50,
    qnode_kwargs=qnode_kwargs,
    atol=0.4,
)

covariance matrix :
 [[ 1.      1.      1.     -0.08    0.08  ]
 [ 1.      1.      1.     -0.08    0.08  ]
 [ 1.      1.      1.     -0.08    0.08  ]
 [-0.08   -0.08   -0.08    0.9856 -0.9856]
 [ 0.08    0.08    0.08   -0.9856  0.9856]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.2786206022533149


## 10 Shots

In [14]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=10,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.72192809 0.3958156  0.97095059 0.3958156  0.55677965]
 [0.3958156  0.46899559 0.97095059 0.55677965 0.3958156 ]
 [0.97095059 0.97095059 0.46899559 0.41997309 0.41997309]
 [0.3958156  0.55677965 0.41997309 0.46899559 1.        ]
 [0.55677965 0.3958156  0.41997309 1.         0.72192809]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.0751390139564574


In [15]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    shots=10,
    qnode_kwargs=qnode_kwargs,
    atol=0.3,
)

covariance matrix :
 [[ 0.96  0.08 -0.24 -0.24  0.24]
 [ 0.08  0.64  0.08  0.08 -0.08]
 [-0.24  0.08  0.96  0.56 -0.56]
 [-0.24  0.08  0.56  0.96 -0.96]
 [ 0.24 -0.08 -0.56 -0.96  0.96]]
decision matrix :
 [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 1 1]
 [0 0 1 1 1]
 [0 0 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.4880514464134382


## 5 Shots


In [16]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=5,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[-0.          0.97095059  0.72192809  0.97095059  0.72192809]
 [ 0.97095059 -0.          0.97095059  0.97095059  0.97095059]
 [ 0.72192809  0.97095059 -0.          0.97095059  0.97095059]
 [ 0.97095059  0.97095059  0.97095059 -0.          0.97095059]
 [ 0.72192809  0.97095059  0.97095059  0.97095059 -0.        ]]
decision matrix :
 [[0 1 1 1 1]
 [1 0 1 1 1]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 0]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  3.9534190384647188


In [17]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    shots=5,
    qnode_kwargs=qnode_kwargs,
    atol=0.5,
)

covariance matrix :
 [[ 0.96  0.16  0.96 -0.16 -0.48]
 [ 0.16  0.96  0.16 -0.96 -0.48]
 [ 0.96  0.16  0.96 -0.16 -0.48]
 [-0.16 -0.96 -0.16  0.96  0.48]
 [-0.48 -0.48 -0.48  0.48  0.64]]
decision matrix :
 [[1 0 1 0 0]
 [0 1 0 1 0]
 [1 0 1 0 0]
 [0 1 0 1 0]
 [0 0 0 0 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.6135033958271414
